In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Carica CIFAR-10
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Classi di CIFAR-10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
               'dog', 'frog', 'horse', 'ship', 'truck']

# Seleziona solo animali e veicoli
animal_classes = [2, 3, 4, 5, 6, 7]  # bird, cat, deer, dog, frog, horse
vehicle_classes = [0, 1, 8, 9]       # airplane, automobile, ship, truck

# Crea nuove etichette: 0=veicolo, 1=animale
def filter_animals_vehicles(X, y):
    idx_animals = np.isin(y, animal_classes).flatten()
    idx_vehicles = np.isin(y, vehicle_classes).flatten()
    idx = idx_animals | idx_vehicles
    X = X[idx]
    y = y[idx]
    y_new = np.where(np.isin(y, animal_classes), 1, 0)
    return X, y_new

X_train_filt, y_train_filt = filter_animals_vehicles(X_train, y_train)
X_test_filt, y_test_filt = filter_animals_vehicles(X_test, y_test)

print("Train set shape:", X_train_filt.shape)
print("Test set shape:", X_test_filt.shape)

In [ ]:
plt.figure(figsize=(10,4))
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.imshow(X_train_filt[i])
    plt.title("Animale" if y_train_filt[i]==1 else "Veicolo")
    plt.axis('off')
plt.suptitle("Esempi dal dataset")
plt.show()

In [ ]:
X_train_filt = X_train_filt.astype('float32') / 255.0
X_test_filt = X_test_filt.astype('float32') / 255.0

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train_filt, y_train_filt, epochs=10, batch_size=64,
                    validation_data=(X_test_filt, y_test_filt))

In [ ]:
plt.plot(history.history['accuracy'], label='Train acc')
plt.plot(history.history['val_accuracy'], label='Val acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Andamento accuratezza')
plt.show()

In [ ]:
# Predizioni
y_pred_prob = model.predict(X_test_filt)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

# Report e confusion matrix
print(classification_report(y_test_filt, y_pred, target_names=['Veicolo', 'Animale']))

cm = confusion_matrix(y_test_filt, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Veicolo','Animale'], yticklabels=['Veicolo','Animale'])
plt.xlabel('Predetto')
plt.ylabel('Reale')
plt.title('Matrice di confusione')
plt.show()

In [ ]:
# Trova immagini sbagliate
wrong_idx = np.where(y_pred != y_test_filt)[0]
plt.figure(figsize=(10,4))
for i, idx in enumerate(wrong_idx[:10]):
    plt.subplot(2,5,i+1)
    plt.imshow(X_test_filt[idx])
    plt.title(f"V:{'Animale' if y_test_filt[idx]==1 else 'Veicolo'}\nP:{'Animale' if y_pred[idx]==1 else 'Veicolo'}")
    plt.axis('off')
plt.suptitle("Esempi di errori di classificazione")
plt.show()